In [4508]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

### Load data

In [4509]:
data_raw = pd.read_csv('ODI-2023.csv', on_bad_lines='skip', delimiter=';')

# Exploratory data analysis

### Create automated pandas report

In [4510]:
# ProfileReport(data_raw)

### What programme are you in?

In [4511]:
#data_raw['What programme are you in?'].value_counts().to_frame()[:30]

In [4512]:
# pd.DataFrame(data_raw['What programme are you in?'].unique())

In [4513]:
# data_raw['What programme are you in?'].str.lower().unique()

In [4514]:
# data_raw[(data_raw['What programme are you in?'].str.contains('AI', case=False)) | (data_raw['What programme are you in?'].str.contains('Artificial Intelligence', case=False))]

### When is your birthday?(date)

In [4515]:
# data_raw['When is your birthday (date)?'].value_counts().to_frame()[:30]

In [4516]:
# data_raw['When is your birthday (date)?'].unique()

### How many students do you estimate are in the room?

In [4517]:
# data_raw['How many students do you estimate there are in the room?'].value_counts().to_frame()[:30]

In [4518]:
# data_raw['How many students do you estimate there are in the room?'].unique()

### What is your stress level?

In [4519]:
# data_raw['What is your stress level (0-100)?'].value_counts().to_frame()[:30]

In [4520]:
# data_raw['What is your stress level (0-100)?'].unique()

### How many hours per week do you do sports? (in whole hours)

In [4521]:
# data_raw['How many hours per week do you do sports (in whole hours)? '].value_counts().to_frame()[:30]

In [4522]:
# data_raw['How many hours per week do you do sports (in whole hours)? '].unique()

### Random number

In [4523]:
# data_raw['Give a random number'].value_counts().to_frame()[:30]

In [4524]:
# data_raw['Give a random number'].unique()

### Time you went to bed Yesterday

In [4525]:
# data_raw['Time you went to bed Yesterday'].value_counts().to_frame()[:30]

In [4526]:
# data_raw['Time you went to bed Yesterday'].unique()

### Correlations investigation

In [4527]:
# data_raw[data_raw['How many hours per week do you do sports (in whole hours)? '] == '2']['What is your stress level (0-100)?']

# Pre processing

In [4528]:
#Fetch column names for renaming

data_raw.columns

Index(['Tijdstempel', 'What programme are you in?',
       'Have you taken a course on machine learning?',
       'Have you taken a course on information retrieval?',
       'Have you taken a course on statistics?',
       'Have you taken a course on databases?', 'What is your gender?',
       'I have used ChatGPT to help me with some of my study assignments ',
       'When is your birthday (date)?',
       'How many students do you estimate there are in the room?',
       'Did you stand up to come to your previous answer    ?',
       'What is your stress level (0-100)?',
       'How many hours per week do you do sports (in whole hours)? ',
       'Give a random number', 'Time you went to bed Yesterday',
       'What makes a good day for you (1)?',
       'What makes a good day for you (2)?'],
      dtype='object')

In [4529]:
#Renaming mapper

renaming_columns = {
    'Tijdstempel' : 'timestamp'
    , 'What programme are you in?': 'programme'
    , 'Have you taken a course on machine learning?' : 'has_ml_knowledge'
    , 'Have you taken a course on information retrieval?' : 'has_ir_knowledge'
    , 'Have you taken a course on statistics?' : 'has_stats_knowledge'
    , 'Have you taken a course on databases?' : 'has_db_knowledge'
    , 'What is your gender?' : 'gender'
    , 'I have used ChatGPT to help me with some of my study assignments ': 'has_used_chatgtp_for_assignments'
    , 'When is your birthday (date)?' : 'birthday'
    , 'How many students do you estimate there are in the room?' : 'no_of_students_estimate'
    , 'Did you stand up to come to your previous answer    ?': 'stood_up_to_estimate_no_of_students'
    , 'What is your stress level (0-100)?' : 'stress_level'
    , 'How many hours per week do you do sports (in whole hours)? ' : 'weekly_sport_hours'
    , 'Give a random number' : 'random_number'
    , 'Time you went to bed Yesterday': 'sleep_time'
    , 'What makes a good day for you (1)?': 'good_day_reason_1'
    , 'What makes a good day for you (2)?': 'good_day_reason_2'
}

In [4530]:
#Perform renaming on dataset

renamed_df = data_raw.rename(columns=renaming_columns)
renamed_df.head(1)

timestamp     programme has_ml_knowledge has_ir_knowledge  \
0  4-4-2023 12:08:33  Masters - AI              yes                0   

  has_stats_knowledge has_db_knowledge        gender  \
0                  mu              nee  gender fluid   

  has_used_chatgtp_for_assignments    birthday no_of_students_estimate  \
0                              yes  23/11/1997                     319   

  stood_up_to_estimate_no_of_students stress_level weekly_sport_hours  \
0                             unknown          100                  0   

  random_number sleep_time good_day_reason_1 good_day_reason_2  
0            69      11:30             Ashes         Not ashes

### Column dropping 

- We will drop timestamp, as an indicator of the time the answer was submitted will not contribute to any of the tasks.
- We will drop has_ir_knowledge as we see high correlation between the has database knowledge and has infomation retrieval knowledge and the latter has less unknowns.
- We will drop birthday as this column is extremely noisy and we believe this will not contribute. Almost all of the students have more or less the same age. In addtion, using a feature on age can be considered non ethical.
- We will drop random number as this is not a significant column.

In [4531]:
#Dropping the columns

renamed_df_new = renamed_df.drop(columns=['timestamp', 'has_ir_knowledge', 'birthday', 'random_number'])
renamed_df_new.head()

programme has_ml_knowledge has_stats_knowledge  \
0                  Masters - AI              yes                  mu   
1       Artificial Intelligence               no               sigma   
2                            AI               no                  mu   
3                   Data mining              yes                  mu   
4  Quantitative Risk Management               no                  mu   

  has_db_knowledge        gender has_used_chatgtp_for_assignments  \
0              nee  gender fluid                              yes   
1              nee          male               not willing to say   
2              nee        female                               no   
3               ja          male                              yes   
4              nee          male                              yes   

  no_of_students_estimate stood_up_to_estimate_no_of_students stress_level  \
0                     319                             unknown          100   
1                     600                                  no           60   
2                     220                                  no           20   
3                     100                             unknown           50   
4                     300                                  no           65   

  weekly_sport_hours sleep_time good_day_reason_1 good_day_reason_2  
0                  0      11:30             Ashes         Not ashes  
1                  2      23.00         good food        good sleep  
2                  5      23:30            Sports           Friends  
3                  5         11               Gym             Piano  
4                 10         11             Relax               Sun

### Cleaning and pre processing feature by feature

### 1) Programme 

We want to map all relevant values to a single label. For example both AI and Artificial Intelligence should be classed as AI.

The sequence of renaming here is very important as certain strings are contained in others and we might clasify certain words wrongly. For example, cs (computer science) is also included in business analyti(cs). Therefore we want to rename business analytics first.

In [4532]:
# Bioinformatics
renamed_df_new.loc[
    (renamed_df_new['programme'].str.contains('bioinformatics', case=False)) 
    | (renamed_df_new['programme'].str.contains('bionformatics', case=False)) #Spelling error
    | (renamed_df_new['programme'].str.contains('bio-informatics', case=False)), 'programme'] = 'Bioinform'

In [4533]:
# Business Analytics
renamed_df_new.loc[
    (renamed_df_new['programme'].str.contains('business analytics', case=False)) 
    | (renamed_df_new['programme'].str.contains('BA', case=True)), 'programme'] = 'BA'

In [4534]:
# Finance and technology
renamed_df_new.loc[
    (renamed_df_new['programme'].str.contains('economics', case=False))
    | (renamed_df_new['programme'].str.contains('finance', case=False)), 'programme'] = 'Finance & Technology'

In [4535]:
# Quantitative Risk Management
renamed_df_new.loc[
    (renamed_df_new['programme'].str.contains('qrm', case=False))
    | (renamed_df_new['programme'].str.contains('quantitative risk management', case=False)), 'programme'] = 'Quantitative Risk Management'

In [4536]:
# Computational science
renamed_df_new.loc[
    (renamed_df_new['programme'].str.contains('cls', case=False))
    | (renamed_df_new['programme'].str.contains('computational', case=False)), 'programme'] = 'Computational science'

In [4537]:
# Data sciecne (econometrics)
renamed_df_new.loc[
    (renamed_df_new['programme'].str.contains('data thingy', case=False))
    | (renamed_df_new['programme'].str.contains('econometrics', case=False)), 'programme'] = 'Data science'

In [4538]:
# Phd
renamed_df_new.loc[(renamed_df_new['programme'].str.contains('phd', case=False)), 'programme'] = 'Phd'

In [4539]:
# Human language
renamed_df_new.loc[(renamed_df_new['programme'].str.contains('human language', case=False)), 'programme'] = 'Human language'

In [4540]:
# Maths
renamed_df_new.loc[(renamed_df_new['programme'].str.contains('mathematics', case=False)), 'programme'] = 'Maths'

In [4541]:
# Other --> Mostly random values
renamed_df_new.loc[
    (renamed_df_new['programme'].str.contains('aryamaan710@gmail.com', case=False))
    | (renamed_df_new['programme'].str.contains('Hyperbolic Time Chamber', case=False))
    | (renamed_df_new['programme'].str.contains('exchange', case=False))
    | (renamed_df_new['programme'].str.contains('Duisenberg fintech', case=False))
    | (renamed_df_new['programme'].str.contains('High school 3rd grade ', case=False))
    | (renamed_df_new['programme'].str.contains('116', case=False)), 'programme'] = 'Other'

In [4542]:
# Artificial intelligence (AI)
renamed_df_new.loc[
    (renamed_df_new['programme'].str.contains('AI', case=False)) 
    | (renamed_df_new['programme'].str.contains('Artifscial Intelligense', case=False)) #Spelling error
    | (renamed_df_new['programme'].str.contains('Artificial Inteligence', case=False))  #Spelling error
    | (renamed_df_new['programme'].str.contains('Artificial Intelligence', case=False)), 'programme'] = 'AI'

In [4543]:
# Computer science
renamed_df_new.loc[
    (renamed_df_new['programme'].str.contains('computer science', case=False)) 
    | (renamed_df_new['programme'].str.contains('big data', case=False)) 
    | (renamed_df_new['programme'].str.contains('cs', case=False)), 'programme'] = 'Computer science'

In [4544]:
# Sanity check
renamed_df_new.programme.unique()

array(['AI', 'Data mining', 'Quantitative Risk Management', 'BA',
       'Computational science', 'Computer science', 'Bioinform', 'Other',
       'Neuroscience', 'Finance & Technology', 'Data science',
       'Human language', 'Masters', 'Master Information Sciences',
       'Biomedical sciences',
       'Management, Policy Analysis and Entrepreneurship in Health and Life Sciences',
       'Master Digital Business and Innovation', 'Phd', 'Maths'],
      dtype=object)

In [4545]:
# Overview
renamed_df_new['programme'].value_counts()

AI                                                                              89
Bioinform                                                                       47
BA                                                                              40
Computer science                                                                37
Data science                                                                    30
Computational science                                                           23
Finance & Technology                                                            14
Other                                                                            7
Quantitative Risk Management                                                     5
Human language                                                                   3
Phd                                                                              2
Neuroscience                                                                     1
Data

### 2) Has ML knowledge

Ideally here we want to replace the unkown values and transform this into a binary feature

In [4546]:
# Overview
renamed_df_new.has_ml_knowledge.value_counts()

yes        221
no          81
unknown      3
Name: has_ml_knowledge, dtype: int64

### 3) Has stats knowledge

In [4547]:
# Renaming to be consistent
renamed_df_new['has_stats_knowledge'] = renamed_df_new['has_stats_knowledge'].replace({'mu':'yes', 'sigma':'no'})

In [4548]:
# Overview
renamed_df_new.has_stats_knowledge.value_counts()

yes        259
no          30
unknown     16
Name: has_stats_knowledge, dtype: int64

### 4) Has databases knowledge

In [4549]:
# Renaming to be consistent
renamed_df_new['has_db_knowledge'] = renamed_df_new['has_db_knowledge'].replace({'ja':'yes', 'nee':'no'})

In [4550]:
# Overview
renamed_df_new.has_db_knowledge.value_counts()

yes        168
no         136
unknown      1
Name: has_db_knowledge, dtype: int64

### 5) Gender

In [4551]:
# Overview
renamed_df_new.gender.value_counts()

male                     154
female                   134
not willing to answer      8
gender fluid               5
non-binary                 3
other                      1
Name: gender, dtype: int64

### 6) Has used chatgtp for assignments

In [4552]:
# Overview
renamed_df_new.has_used_chatgtp_for_assignments.value_counts()

yes                   192
no                     73
not willing to say     40
Name: has_used_chatgtp_for_assignments, dtype: int64

### 7) Number of students estimate	

Here we replace the text inputs with numerical values, then translate str to int and apply a trasformation to deal with outliers and missing values

In [4553]:
# Overview
renamed_df_new.no_of_students_estimate.unique()

array(['319', '600', '220', '100', '300', '500', '150', '525', '570',
       '400', '550', '-6382', '450', '230', '750', '534', '350', '800',
       '516', '85', '380', '200', '449', '270', '180', '520', '75', '581',
       '120', '222', '580', '285', '170', '365', '555', '226', '280',
       'six hundred', '175', '250', '284', '453', '31', '487', '187',
       '236', '229', '420', '427', '456', '900', '275', '376', '300.5',
       '413', '325', '543', '575', '287', '283', '334', '657', '348',
       '189', '477', '451', '438', 'Over 9000', '266', '643', '458',
       '588', '410', '418', '339', '473', '332', '140', '183', '90',
       '509', '290', '557', '298', '387', '386', '228', '260', '326',
       '436', '305', '312', '623', '205', '567', '340', '421',
       'Bout six hunnid', '201', '369', '428', 'Around 200', '500+',
       '224', '476', '198', '69,420/  '], dtype=object)

In [4554]:
# Replace all noise with values that can be mapped to ints
renamed_df_new.loc[
    (renamed_df_new['no_of_students_estimate'].str.contains('Bout six hunnid', case=False))
    | (renamed_df_new['no_of_students_estimate'].str.contains('six hundred', case=False)), 'no_of_students_estimate'] = '600'

In [4555]:
renamed_df_new.loc[(renamed_df_new['no_of_students_estimate'].str.contains('Around 200', case=False)), 'no_of_students_estimate'] = '200'

In [4556]:
renamed_df_new.loc[(renamed_df_new['no_of_students_estimate'].str.contains('300.5', case=False)), 'no_of_students_estimate'] = '301'

In [4557]:
renamed_df_new.loc[(renamed_df_new['no_of_students_estimate'].str.contains('500+', case=False)), 'no_of_students_estimate'] = '550'

In [4558]:
renamed_df_new.loc[(renamed_df_new['no_of_students_estimate'].str.contains('Over 9000', case=False)), 'no_of_students_estimate'] = '9000'

In [4559]:
renamed_df_new.loc[
    (renamed_df_new['no_of_students_estimate'].str.contains('69,420/  ', case=False))
    ,'no_of_students_estimate'] = np.nan

In [4560]:
# Overview after first cleaning
renamed_df_new.no_of_students_estimate.unique()

array(['319', '600', '220', '100', '300', '550', '150', '525', '570',
       '400', '-6382', '450', '230', '750', '534', '350', '800', '516',
       '85', '380', '200', '449', '270', '180', '520', '75', '581', '120',
       '222', '580', '285', '170', '365', '555', '226', '280', '175',
       '250', '284', '453', '31', '487', '187', '236', '229', '420',
       '427', '456', '900', '275', '376', '301', '413', '325', '543',
       '575', '287', '283', '334', '657', '348', '189', '477', '451',
       '438', '9000', '266', '643', '458', '588', '410', '418', '339',
       '473', '332', '140', '183', '90', '509', '290', '557', '298',
       '387', '386', '228', '260', '326', '436', '305', '312', '623',
       '205', '567', '340', '421', '201', '369', '428', '224', '476',
       '198', nan], dtype=object)

In [4561]:
renamed_df_new['no_of_students_estimate'] = pd.to_numeric(renamed_df_new['no_of_students_estimate'], errors='coerce')

In [4562]:
# Mapping to numerical values 
renamed_df_new['no_of_students_estimate'] = renamed_df_new['no_of_students_estimate'].astype('Int64')

In [4563]:
# Treating outliers using quantile information 
lower_percentile = round(renamed_df_new.no_of_students_estimate.quantile(0.01))
upper_percentile = round(renamed_df_new.no_of_students_estimate.quantile(0.99))
renamed_df_new['no_of_students_estimate'].clip(lower_percentile, upper_percentile, inplace=True)

In [4564]:
# Replacing null value with mean 
renamed_df_new['no_of_students_estimate'] = renamed_df_new.no_of_students_estimate.fillna(renamed_df_new['no_of_students_estimate'].median())

In [4565]:
renamed_df_new.no_of_students_estimate.unique()

<IntegerArray>
[319, 600, 220, 100, 300, 550, 150, 525, 570, 400,  85, 450, 230, 750, 534,
 350, 798, 516, 380, 200, 449, 270, 180, 520, 581, 120, 222, 580, 285, 170,
 365, 555, 226, 280, 175, 250, 284, 453, 487, 187, 236, 229, 420, 427, 456,
 275, 376, 301, 413, 325, 543, 575, 287, 283, 334, 657, 348, 189, 477, 451,
 438, 266, 643, 458, 588, 410, 418, 339, 473, 332, 140, 183,  90, 509, 290,
 557, 298, 387, 386, 228, 260, 326, 436, 305, 312, 623, 205, 567, 340, 421,
 201, 369, 428, 224, 476, 198]
Length: 96, dtype: Int64

### 8) Stood up to estimate


In [4566]:
# Overview
renamed_df_new.stood_up_to_estimate_no_of_students.value_counts()

no         276
yes         17
unknown     12
Name: stood_up_to_estimate_no_of_students, dtype: int64

### 9) Stress level (0-100)
Here we want to again end up with a numerical feature. Therefore, we will clean some noise and then make sure to treat outliers. In general, the approach followed for outliers here is anything above 100 will be truncated to 100 and anything below 0 to 0. For values we are uncertain we will mark them as null and replace them using a (add technique here).

In [4567]:
# Overview
renamed_df_new.stress_level.unique()

array(['100', '60', '20', '50', '65', '80', '30', '70', '10', '-100',
       '15', '42', '0', '77', '40', '6', '99', '67', '90', '106', '39',
       '25', '55', '999', '35', '17', '98', '4274', '5', '66', '51',
       '990.000.001', '75', '100 always', '9', '86', '85', '69', '68',
       '43', '1', '22', '62', '101', '74', '45', '65?', '23', '95', '8',
       '-5', '26', '50.321', '7', '1E+11', '73', nan, '11', '18', '31',
       '33', '99.9', '-420', 'At this moment my stress level is 21',
       '200', '2', '88', 'What grazia said', 'Np.inf', 'Inf ', '53',
       'solid 0.2', '44', '56', '57', '28', 'it is what it is'],
      dtype=object)

In [4568]:
# Replace all noise with values that can be mapped to ints

renamed_df_new.loc[(renamed_df_new['stress_level'].str.contains('990.000.001', case=False, na=False)), 'stress_level'] = '990'

In [4569]:
renamed_df_new.loc[(renamed_df_new['stress_level'].str.contains('At this moment my stress level is 21', case=False, na=False)), 'stress_level'] = '21'

In [4570]:
renamed_df_new.loc[(renamed_df_new['stress_level'].str.contains('solid 0.2', case=False, na=False)), 'stress_level'] = '0'

In [4571]:
renamed_df_new.loc[(renamed_df_new['stress_level'].str.contains('50.321', case=False, na=False)), 'stress_level'] = '50'

In [4572]:
renamed_df_new.loc[(renamed_df_new['stress_level'].str.contains('99.9', case=False, na=False)), 'stress_level'] = '100'

In [4573]:
renamed_df_new.loc[(renamed_df_new['stress_level'].str.contains('65?', case=False, na=False)), 'stress_level'] = '65'

In [4574]:
renamed_df_new.loc[
    (renamed_df_new['stress_level'].str.contains('What grazia said', case=False))
    | (renamed_df_new['stress_level'].str.contains('it is what it is', case=False)), 'stress_level'] = '100'

In [4575]:
renamed_df_new.loc[
    (renamed_df_new['stress_level'].str.contains('Np.inf', case=False))
    | (renamed_df_new['stress_level'].str.contains('E', case=True))
    | (renamed_df_new['stress_level'].str.contains('100 always', case=False))
    | (renamed_df_new['stress_level'].str.contains('Inf ', case=False)), 'stress_level'] = '100'
    

In [4576]:
renamed_df_new.stress_level.unique()

array(['100', '65', '20', '50', '80', '30', '70', '10', '-100', '15',
       '42', '0', '77', '40', '99', '90', '39', '25', '55', '999', '35',
       '17', '98', '4274', '5', '51', '990', '75', '9', '85', '43', '1',
       '22', '101', '74', '45', '23', '95', '8', '-5', '7', '73', nan,
       '11', '18', '31', '33', '-420', '21', '200', '2', '88', '53', '44',
       '57', '28'], dtype=object)

In [4577]:
print(renamed_df_new['stress_level'].dtype)
renamed_df_new['stress_level'] = pd.to_numeric(renamed_df_new['stress_level'], errors='coerce')

object


In [4578]:
# Mapping to numerical values 
renamed_df_new['stress_level'] = renamed_df_new['stress_level'].astype('Int64')

In [4579]:
# Truncating extreme values between 0-100.
renamed_df_new['stress_level'].clip(0, 100, inplace=True)

In [4580]:
# Using K nearest neighbour approach to impute missing value
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)

renamed_df_new['stress_level'] = imputer.fit_transform(renamed_df_new[['stress_level']]).round()

### 10) Weekly sport hours

In [4581]:
# Overview
renamed_df_new.weekly_sport_hours.unique()

array(['0', '2', '5', '10', '7', '4', '6', '0.5', 'Nadda ', '3', '3-4',
       '12', '8', '290', '1', '-10', '5-6', '15', '9', '4/5', '11',
       '3hours', '1 hour', '1,5', '20', '0.01', '7h', '21', '8-10',
       '4 hours', 'h', 'Y = 5x + 3 ', 'Np.nan ', '3 hrs', '3,5',
       'depends on how many hours I spent with your mom'], dtype=object)

In [4582]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('7h', case=False, na=False)), 'weekly_sport_hours'] = '7'

In [4583]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('0.5', case=False, na=False)), 'weekly_sport_hours'] = '1'

In [4584]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('1,5', case=False, na=False)), 'weekly_sport_hours'] = '2'

In [4585]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('3,5', case=False, na=False)), 'weekly_sport_hours'] = '4'

In [4586]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('1 hour', case=False, na=False)), 'weekly_sport_hours'] = '1'

In [4587]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('4 hour', case=False, na=False)), 'weekly_sport_hours'] = '4'

In [4588]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('3hours', case=False, na=False)), 'weekly_sport_hours'] = '3'

In [4589]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('3 hrs', case=False, na=False)), 'weekly_sport_hours'] = '3'

In [4590]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('Nadda', case=False, na=False)), 'weekly_sport_hours'] = '0'

In [4591]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('0.01', case=False, na=False)), 'weekly_sport_hours'] = '0'

In [4592]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('3-4', case=False, na=False)), 'weekly_sport_hours'] = '3'

In [4593]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('5-6', case=False, na=False)), 'weekly_sport_hours'] = '6'

In [4594]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('8-10', case=False, na=False)), 'weekly_sport_hours'] = '9'

In [4595]:
renamed_df_new.loc[(renamed_df_new['weekly_sport_hours'].str.contains('4/5', case=False, na=False)), 'weekly_sport_hours'] = '4'

In [4596]:
renamed_df_new.loc[
    (renamed_df_new['weekly_sport_hours'].str.contains('depends on how many hours I spent with your mom', case=False))
    | (renamed_df_new['weekly_sport_hours'].str.contains('Np.nan ', case=False))
    | (renamed_df_new['weekly_sport_hours'].str.contains('Y', case=True))
    | (renamed_df_new['weekly_sport_hours'].str.contains('h', case=False)), 'weekly_sport_hours'] = np.nan

In [4597]:
renamed_df_new['weekly_sport_hours'] = pd.to_numeric(renamed_df_new['weekly_sport_hours'], errors='coerce')

In [4598]:
# Mapping to numerical values 
renamed_df_new['weekly_sport_hours'] = renamed_df_new['weekly_sport_hours'].astype('Int64')

In [4599]:
# Treating outliers using quantile information 
lower_percentile = round(renamed_df_new.weekly_sport_hours.quantile(0.01))
upper_percentile = round(renamed_df_new.weekly_sport_hours.quantile(0.98))
renamed_df_new['weekly_sport_hours'].clip(lower_percentile, upper_percentile, inplace=True)

In [4600]:
renamed_df_new['weekly_sport_hours'] = renamed_df_new['weekly_sport_hours'].fillna(renamed_df_new['weekly_sport_hours'].median())

### 11) Sleep time

In [4601]:
# Overview
renamed_df_new.sleep_time.unique()
renamed_df_new.sleep_time.unique()

array(['11:30', '23.00', '23:30', '11', '12am', '23:00', '00:00', '2230',
       '00:15', '7：00', '4 am ', '02:00', '4:30', '22:45', '12:30',
       '22:30', '4am', '12:00am', '12.00', '00:30', '12', '02.00', '2 am',
       '2:45', '01:00', '22:20', '24:00', '11.30', '2 am ', '6 am',
       '4:00am', 'after midnight', '1am', '0 am', '1230', '12:00',
       '00.30', '0000', '12:30am', '1:40', '22:00', '01.00', '0:15',
       'around 5:30', '1 am', '1:00', '23.40', '2300', '11 PM', '22.00',
       '12:15', '2am', '24', '21:30', '23:59', 'late ', '3', '02.35',
       '23', '23:15', '22:50', '1:20 am', '23.15', '1:00 AM ', '3 am',
       '00:25', '00:20', '23.59', '0030', '23:45', '22.30', '22.16',
       '12 am', '21:53', '2:00', '2:30am', 'Midnight', '01:00 ', '0200',
       '23.30', '00:22', '02:45', '23u', '21.00', '23.58', '12:30 am',
       '23.5', '00.26', '11,1', '2', '2AM', '21:00', '2 AM', '1',
       'Did not slept on the couch', '23:50', '02:00 am',
       'When the moon was at

In [4602]:
mapper = {
'11:30':'pm', 
'23.00':'pm', 
'23:30':'pm', 
'11':'pm', 
'12am':'am', 
'23:00':'pm', 
'00:00':'am', 
'2230':'pm',
'00:15':'am', 
'7:00':'am', 
'4 am ':'am', 
'02:00':'am', 
'4:30':'am', 
'22:45':'pm', 
'12:30':'am',
'22:30':'pm', 
'4am':'am', 
'12:00am':'am', 
'12.00':'am', 
'00:30':'am', 
'12':'am', 
'02.00':'am', 
'2 am':'am',
'2:45':'am', 
'01:00':'am', 
'22:20':'pm', 
'24:00':'am', 
'11.30':'pm', 
'2 am ':'am', 
'6 am':'am',
'4:00am':'am', 
'after midnight':'am', 
'1am':'am', 
'0 am':'am', 
'1230':'am', 
'12:00':'am',
'00.30':'am', 
'0000':'am', 
'12:30am':'am', 
'1:40':'am', 
'22:00':'pm', 
'01.00':'am', 
'0:15':'am',
'around 5:30':'am', 
'1 am':'am', 
'1:00':'am', 
'23.40':'pm', 
'2300':'pm', 
'11 PM':'pm', 
'22.00':'pm',
'12:15':'am', 
'2am':'am', 
'24':'am', 
'21:30':'pm', 
'23:59':'pm', 
'late ':'am', 
'3':'am', 
'02.35':'am',
'23':'pm', 
'23:15':'pm', 
'22:50':'pm', 
'1:20 am':'am', 
'23.15':'pm', 
'1:00 AM ':'am', 
'3 am':'am',
'00:25':'am', 
'00:20':'am', 
'23.59':'pm', 
'0030':'am', 
'23:45':'pm', 
'22.30':'pm', 
'22.16':'pm',
'12 am':'am', 
'21:53':'pm', 
'2:00':'am', 
'2:30am':'am', 
'Midnight':'pm',
'01:00 ':'am', 
'0200':'am',
'23.30':'pm', 
'00:22':'am', 
'02:45':'am', 
'23u':'pm', 
'21.00':'pm', 
'23.58':'pm', 
'12:30 am':'am',
'23.5':'pm', 
'00.26':'am', 
'11,1':'pm', 
'2':'am', 
'2AM':'am', 
'21:00':'pm', 
'2 AM':'am', 
'1':'am',
'Did not slept on the couch':np.nan, 
'23:50':'pm', 
'02:00 am':'am',
'When the moon was at the highest point':'am', 
'2:01':'am', 
'00:40':'am', 
'01:30':'am',
'2.00':'am', 
'01:50':'am', 
'11pm':'pm', 
'1:30':'am', 
'4 hours':'am', 
'3:00':'am', 
'late':'am',
'half past twelve':'am', 
'03:00':'am', 
'22.53':'pm', 
'I went to bed at 1':'am',
'I didnt ':np.nan, 
'23:20':'pm', 
'23.17':'pm', 
'Non of ya business':np.nan, 
'12:00AM':'am',
'2:30':'am', 
'3am':'am', 
'2.30':'am', 
'1pm':'am', 
'01:09':'am', 
'4:00':'am',
'after your dad went ':np.nan, 
'4:30am':'am'
}


In [4603]:
renamed_df_new = renamed_df_new.replace(mapper)

In [4604]:
renamed_df_new

programme has_ml_knowledge has_stats_knowledge  \
0                              AI              yes                 yes   
1                              AI               no                  no   
2                              AI               no                 yes   
3                     Data mining              yes                 yes   
4    Quantitative Risk Management               no                 yes   
..                            ...              ...                 ...   
300                         Other               no             unknown   
301                     Bioinform              yes                 yes   
302                            AI              yes                 yes   
303                            AI              yes                 yes   
304                  Data science               no                 yes   

    has_db_knowledge        gender has_used_chatgtp_for_assignments  \
0                 no  gender fluid                              yes   
1                 no          male               not willing to say   
2                 no        female                               no   
3                yes          male                              yes   
4                 no          male                              yes   
..               ...           ...                              ...   
300              yes          male                               no   
301               no        female                               no   
302              yes  gender fluid                               no   
303               no          male               not willing to say   
304              yes        female                               no   

     no_of_students_estimate stood_up_to_estimate_no_of_students  \
0                        319                             unknown   
1                        600                                  no   
2                        220                                  no   
3                        100                             unknown   
4                        300                                  no   
..                       ...                                 ...   
300                      400                                  no   
301                      150                                  no   
302                      350                                 yes   
303                      100                                  no   
304                      200                                 yes   

     stress_level  weekly_sport_hours sleep_time          good_day_reason_1  \
0           100.0                   0         pm                      Ashes   
1            65.0                   2         pm                  good food   
2            20.0                   5         pm                     Sports   
3            50.0                   5         pm                        Gym   
4            65.0                  10         pm                      Relax   
..            ...                 ...        ...                        ...   
300          70.0                   0         am                Productive    
301          10.0                   5         am                Nice people   
302         100.0                   4        NaN  never before seen trauma    
303          50.0                   1         am                       Work   
304          80.0                   3         pm                  Good meal   

               good_day_reason_2  
0                      Not ashes  
1                     good sleep  
2                        Friends  
3                          Piano  
4                            Sun  
..                           ...  
300                  Successful   
301  Educational accomplishments  
302                       you :)  
303                Relationships  
304                        Sport  

[305 rows x 13 columns]

### 12) What makes you feel good 1+2

In [4605]:
#Mapping to string type
renamed_df_new['good_day_reason_1'] = renamed_df_new['good_day_reason_1'].astype(str)
renamed_df_new['good_day_reason_2'] = renamed_df_new['good_day_reason_2'].astype(str)

In [4606]:
#Create two new columns for categories of good_day_reason_1 and good_day_reason_2
renamed_df_new["good_day_reason_1_category"] = renamed_df_new["good_day_reason_1"].copy()
renamed_df_new["good_day_reason_2_category"] = renamed_df_new["good_day_reason_2"].copy()

In [4607]:
# Overview - good_day_reason_1
renamed_df_new.good_day_reason_1.unique()

array(['Ashes', 'good food', 'Sports', 'Gym', 'Relax', 'Coffee',
       'Good code', 'When the sun shines', 'Expierence ',
       'Experience a serotonin molecule ', 'You', 'delicious food',
       'Netflix and', 'Something surprising emerging', 'Coffee ', 'Sun',
       'Sunny weather', 'Thursday ', 'Sunshine', 'Good food ',
       'Getting things done', 'No task', 'eat well', 'The sun',
       'Good food', 'Eating chocolate', 'Being productive ', 'food ',
       'Good night rest', 'Family ', 'blue skies', 'Good weather',
       'sunny days', 'Slept well', 'Friends', 'A sunny day ', 'Sunny',
       'Feeling productive', 'No obligations', 'good morning routine',
       'Family!', 'Good weather ', 'food', 'bread', 'coffee',
       'Sticking to my timetable ', 'The sun ', 'Working out',
       'Nice weather', 'Good results', 'No class', 'Good breakfast',
       'A day where I went to the gym', 'Food', 'Vacation',
       'good weather ', 'Good meal', 'Music', 'No lecture', 'Holiday ',
    

In [4608]:
# Creating categories for good_day_reason_1 good_day_reason_1_category

for i, row in renamed_df_new.iterrows():
    
    # Food and drinks category
    if "coffee" in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'food' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'sausage' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif '☕' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'cake' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'bier' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'eat' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'chocolate' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'meal' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'lunch' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'fed' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'fed' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'beer' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'bread' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'eten' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'meals' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'breakfast' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'oatmeal' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'drinks' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'sushi' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'avocado' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'pizza' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'eating' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'foody' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'pancakes' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'snacks' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'alcohol' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    elif 'diner' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "food and drinks"
    
    # Weather category    
    elif 'weather' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "weather"
    elif 'sun' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "weather"
    elif 'sunny' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "weather"
    elif 'sunshine' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "weather"
    elif 'skies' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "weather"
        
    # Sport category
    elif 'sports' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'gym' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'sport' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'workout' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'exercised' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'working out' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'exercise' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'worked out' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'training' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'basketball' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'excercise' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'excersise' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    elif 'activity' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sport"
    
    # Sleep and rest category        
    elif 'sleeping' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sleep and rest"
    elif 'sleep' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sleep and rest"
    elif 'night' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sleep and rest"
    elif 'slept' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sleep and rest"
    elif 'rested' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sleep and rest"
    elif 'slaap' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "sleep and rest"
        
    # Social interaction category    
    elif 'friends' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'family' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'company' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'people' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'friend' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'mensen' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'socializing' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'loved' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'hugs' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'love' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'person' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'communication' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    elif 'relationships' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "social interaction"
    
    # Study-related category
    elif 'class' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'study' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'grade' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'task' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'assignment' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'results' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'exam' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'lectures' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'lecture' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'school' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'deadlines' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'exams' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'grades' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'educational' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'credits' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'course' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
    elif 'ddl' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "study-related"
        
    # Leisure category  
    elif 'netflix' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'game' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'biking' in row["good_day_reason_1"].lower():                    
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'music' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'movie' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'watch' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'book' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'watching' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'piano' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'travelling' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'yoga' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'painting' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'read' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'activities' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"   
    elif 'chill' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'walking' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'walk' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'fun' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'holiday' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'travel' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'exploring' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'chess' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'playing' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'church' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'vacation' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'beach' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'relaxing' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'relax' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'sex' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'outside' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    elif 'outisde' in row["good_day_reason_1"].lower():
        renamed_df_new.at[i, "good_day_reason_1_category"] = "leisure"
    
    # Others category 
    else:
        renamed_df_new.at[i, "good_day_reason_1_category"] = "others"


In [4609]:
renamed_df_new.good_day_reason_1_category.unique()

array(['others', 'food and drinks', 'sport', 'leisure', 'weather',
       'study-related', 'sleep and rest', 'social interaction'],
      dtype=object)

In [4610]:
# Overview - good_day_reason_2
renamed_df_new.good_day_reason_2.unique()

array(['Not ashes', 'good sleep', 'Friends', 'Piano', 'Sun',
       'Sunny weather', 'Good code', 'If I am able to sport',
       'Expierence', 'Experience another serotonin molecule ',
       'Finally losing my virginity', 'nothing', 'Chill',
       'Nothing bad happened', 'Engaging activities ', 'Food',
       'A good night sleep', 'Going to the gym', 'Saturday',
       'Friends or family ', 'Good company', 'Creating', 'Alone',
       'sleep well', 'Friends and food', 'Productivity',
       'Getting a high grade ', 'Having a good meal', 'game',
       'No anxiety', 'Good food', 'lakes/beach', 'No assigment',
       'good meals', 'Food ', 'Family', 'If I had a nice workout ',
       'Good food ', 'Exercised', 'Nice people',
       'Being outside in the sun', 'sunny weather', 'Friends!', 'Cats ',
       'sex', 'coffee', 'Eating good food ', 'Chill in nature',
       'No class', 'Kindness', 'Prodictivity', 'sunset', 'Biking',
       'No exam', 'Good workout', 'A productive day study-wis

In [4611]:
# Creating categories for good_day_reason_2 in good_day_reason_2_category

for i, row in renamed_df_new.iterrows():
    
    # Food and drinks category
    if "coffee" in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'food' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'sausage' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif '☕' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'cake' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'bier' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'eat' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'chocolate' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'meal' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'lunch' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'fed' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'fed' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'beer' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'bread' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'eten' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'meals' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'breakfast' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'oatmeal' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'drinks' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'sushi' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'avocado' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'pizza' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'eating' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'foody' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'pancakes' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'snacks' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'alcohol' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
    elif 'diner' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "food and drinks"
        
    # Weather category    
    elif 'weather' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "weather"
    elif 'sun' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "weather"
    elif 'sunny' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "weather"
    elif 'sunshine' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "weather"
    elif 'skies' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "weather"
        
    # Sport category
    elif 'sports' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'gym' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'sport' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'workout' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'exercised' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'working out' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'exercise' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'worked out' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'training' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'basketball' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'excercise' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'excersise' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
    elif 'activity' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sport"
        
    # Sleep and rest category   
    elif 'sleeping' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sleep and rest"
    elif 'sleep' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sleep and rest"
    elif 'night' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sleep and rest"
    elif 'slept' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sleep and rest"
    elif 'rested' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sleep and rest"
    elif 'slaap' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "sleep and rest"
        
        
    # Social interaction category    
    elif 'friends' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'family' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'company' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'people' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'friend' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'mensen' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'socializing' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'loved' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'hugs' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'love' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'person' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'communication' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    elif 'relationships' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "social interaction"
    
    
    # Study-related category
    elif 'class' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'study' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'grade' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'task' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'assignment' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'assigment' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'results' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'exam' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'lectures' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'lecture' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'school' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'deadlines' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'exams' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'grades' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'educational' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'credits' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'course' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
    elif 'ddl' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "study-related"
        
    
    # Leisure category
    elif 'netflix' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'game' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'biking' in row["good_day_reason_2"].lower():                    
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'music' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'movie' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'watch' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'book' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'watching' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'piano' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'travelling' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'yoga' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'painting' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'read' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'activities' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'chill' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'walking' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'walk' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'fun' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'holiday' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'travel' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'exploring' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'chess' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'playing' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'church' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'vacation' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'beach' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'relaxing' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'relax' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'sex' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'outside' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"
    elif 'outisde' in row["good_day_reason_2"].lower():
        renamed_df_new.at[i, "good_day_reason_2_category"] = "leisure"


    # Others category
    else:
        renamed_df_new.at[i, "good_day_reason_2_category"] = "others"


In [4612]:
renamed_df_new.good_day_reason_2_category.unique()

array(['others', 'sleep and rest', 'social interaction', 'leisure',
       'weather', 'food and drinks', 'sport', 'study-related'],
      dtype=object)

In [4613]:
renamed_df_new

programme has_ml_knowledge has_stats_knowledge  \
0                              AI              yes                 yes   
1                              AI               no                  no   
2                              AI               no                 yes   
3                     Data mining              yes                 yes   
4    Quantitative Risk Management               no                 yes   
..                            ...              ...                 ...   
300                         Other               no             unknown   
301                     Bioinform              yes                 yes   
302                            AI              yes                 yes   
303                            AI              yes                 yes   
304                  Data science               no                 yes   

    has_db_knowledge        gender has_used_chatgtp_for_assignments  \
0                 no  gender fluid                              yes   
1                 no          male               not willing to say   
2                 no        female                               no   
3                yes          male                              yes   
4                 no          male                              yes   
..               ...           ...                              ...   
300              yes          male                               no   
301               no        female                               no   
302              yes  gender fluid                               no   
303               no          male               not willing to say   
304              yes        female                               no   

     no_of_students_estimate stood_up_to_estimate_no_of_students  \
0                        319                             unknown   
1                        600                                  no   
2                        220                                  no   
3                        100                             unknown   
4                        300                                  no   
..                       ...                                 ...   
300                      400                                  no   
301                      150                                  no   
302                      350                                 yes   
303                      100                                  no   
304                      200                                 yes   

     stress_level  weekly_sport_hours sleep_time          good_day_reason_1  \
0           100.0                   0         pm                      Ashes   
1            65.0                   2         pm                  good food   
2            20.0                   5         pm                     Sports   
3            50.0                   5         pm                        Gym   
4            65.0                  10         pm                      Relax   
..            ...                 ...        ...                        ...   
300          70.0                   0         am                Productive    
301          10.0                   5         am                Nice people   
302         100.0                   4        NaN  never before seen trauma    
303          50.0                   1         am                       Work   
304          80.0                   3         pm                  Good meal   

               good_day_reason_2 good_day_reason_1_category  \
0                      Not ashes                     others   
1                     good sleep            food and drinks   
2                        Friends                      sport   
3                          Piano                      sport   
4                            Sun                    leisure   
..                           ...                        ...   
300                  Successful                      others   
301  Educati

In [4614]:
renamed_df_new['good_day_reason_1_category'].value_counts()

food and drinks       97
others                65
weather               54
leisure               22
sport                 20
sleep and rest        19
social interaction    16
study-related         12
Name: good_day_reason_1_category, dtype: int64

In [4615]:
renamed_df_new['good_day_reason_2_category'].value_counts()

food and drinks       70
others                62
leisure               46
social interaction    41
weather               25
sleep and rest        22
sport                 20
study-related         19
Name: good_day_reason_2_category, dtype: int64

In [4616]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

## configurations
#### classification or regression?
TASK = "prediction"

## start from the last line in the notebook
## drop the two unused columns
data = renamed_df_new.drop(columns = ["good_day_reason_1", "good_day_reason_2"])

# Create a StandardScaler object
scaler = StandardScaler()

# Scale the 'no_of_students_estimate' and 'weekly_sport_hours' columns
data[['no_of_students_estimate', 'weekly_sport_hours']] = scaler.fit_transform(data[['no_of_students_estimate', 'weekly_sport_hours']])

## build the stress label, which is used for classification
data["stress_label"] = pd.cut(data["stress_level"], [-1, 40, 70, 101], labels = ["good", "normal", "stressed"])

## split the target columns and features
clf_tgt = data["stress_label"]
rgs_tgt = data["stress_level"]
data = data.drop(columns = ["stress_label", "stress_level"])

## one-hot encoding for string columns
data = pd.get_dummies(data)

## finalizing pre-processing
data["stress_level"] = rgs_tgt
data["stress_label"] = clf_tgt

## write the result to a csv
data.to_csv("data.csv", index = False)

## free memories
del clf_tgt
del rgs_tgt
del renamed_df_new
del data

## read the data
data = pd.read_csv("data.csv")

In [4617]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV

TASK = 'classification'

## prepare for machine learning
if TASK == "classification":
    y = data["stress_label"].to_numpy()
else:
    y = data["stress_level"].to_numpy()

X = data.drop(columns = ["stress_label", "stress_level"]).to_numpy()

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Define number of folds
num_folds = 4

# Define cross-validation method (KFold or StratifiedKFold)
cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [4618]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Define base estimator and AdaBoost model
base_estimator = DecisionTreeClassifier(max_depth=1)
model = AdaBoostClassifier(base_estimator=base_estimator, random_state=42)

# Define parameter grid for grid search
param_grid = {'n_estimators': [50, 100, 200],
              'learning_rate': [0.1, 0.5, 1.0],
              'base_estimator__max_depth': [1, 2, 3]
}

# Initialize grid search
grid_search = GridSearchCV(model, param_grid=param_grid, cv=cv)

# Fit grid search on training data
grid_search.fit(X_train, y_train)

# Print best parameters and corresponding score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Evaluate best model on test set
best_model = grid_search.best_estimator_
feature_importances = best_model.feature_importances_
test_score = best_model.score(X_test, y_test)

print("Test score", test_score)
# print("Feature importances", feature_importances)

Best parameters:  {'base_estimator__max_depth': 3, 'learning_rate': 1.0, 'n_estimators': 50}
Best score:  0.45987851662404094
Test score 0.45161290322580644


In [4619]:
# X_new = pd.DataFrame(X).drop(pd.DataFrame(X).columns[np.where(feature_importances == 0)[0].tolist()], axis=1).to_numpy()

# # Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.1, random_state=42)

# # Define number of folds
# num_folds = 4

# # Define cross-validation method (KFold or StratifiedKFold)
# cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# # Define base estimator and AdaBoost model
# base_estimator = DecisionTreeClassifier(max_depth=1)
# model = AdaBoostClassifier(base_estimator=base_estimator, random_state=42)

# # Define parameter grid for grid search
# param_grid = {'n_estimators': [50, 100, 200],
#               'learning_rate': [0.1, 0.5, 1.0],
#               'base_estimator__max_depth': [1, 2, 3]
# }

# # Initialize grid search
# grid_search = GridSearchCV(model, param_grid=param_grid, cv=cv)

# # Fit grid search on training data
# grid_search.fit(X_train, y_train)

# # Print best parameters and corresponding score
# print("Best parameters: ", grid_search.best_params_)
# print("Best score: ", grid_search.best_score_)

# # Evaluate best model on test set
# best_model = grid_search.best_estimator_
# test_score_new = best_model.score(X_test, y_test)

# print("Test score: ", test_score_new)
# print(f"After dropping non important features the accuracy increase by {round((test_score_new - test_score) / test_score * 100, 1)}%" )

In [4620]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Define parameter grid for grid search
param_grid = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7],
              'weights': ['uniform', 'distance'],
              'p': [1, 2],
              # 'leaf_size': [10, 30, 60, 90],
            #   'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Create a KNeighborsClassifier object
knn = KNeighborsClassifier()

# Define number of folds
num_folds = 4

# Define cross-validation method (KFold or StratifiedKFold)
cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize grid search
grid_search = GridSearchCV(knn, param_grid=param_grid, cv=cv)

# Fit grid search on training data
grid_search.fit(X_train, y_train)

# Print best parameters and corresponding score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Evaluate best model on test set
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Test score", accuracy)

Best parameters:  {'n_neighbors': 2, 'p': 2, 'weights': 'uniform'}
Best score:  0.45268542199488493
Test score 0.3870967741935484


In [4621]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV

TASK = 'regression'

## prepare for machine learning
if TASK == "classification":
    y = data["stress_label"].to_numpy()
else:
    y = data["stress_level"].to_numpy()

X = data.drop(columns = ["stress_label", "stress_level"]).to_numpy()

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Define number of folds
num_folds = 3

# Define cross-validation method (KFold or StratifiedKFold)
cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [4623]:
param_grid = {
    'alpha' : [0.1, 0.2, 0.4, 0.5, 1.0]
}

model = Lasso(random_state=42)

# Initialize grid search
grid_search = GridSearchCV(model, param_grid=param_grid, cv=cv)

# Fit grid search on training data
grid_search.fit(X_train, y_train)

# Print best parameters and corresponding score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Evaluate best model on test set
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)

mse = mean_squared_error(y_test, best_model.predict(X_test))
mae = mean_absolute_error(y_test, best_model.predict(X_test))

print("R2:", test_score)
print("MSE:", mse )
print("MAE:", mae )

Best parameters:  {'alpha': 0.4}
Best score:  0.018940319816606006
R2: 0.0449633587851761
MSE: 684.6748116180572
MAE: 21.962838923330924


/opt/homebrew/Caskroom/miniforge/base/envs/main/lib/python3.8/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
